## Preprocess training data to BERT's format. 
### we will get train.tsv and dev.tsv

In [1]:
import random
random.seed = 16
import pandas as pd
from gensim.models.word2vec import Word2Vec
import jieba
from collections import Counter
import numpy as np
import os
import pickle
import re
print("import ended...")

import ended...


In [2]:
print("read csv started...")
data = pd.read_csv("ai_challenger_sentiment_analysis_trainingset_20180816/sentiment_analysis_trainingset.csv")
data_validation = pd.read_csv("ai_challenger_sentiment_analysis_validationset_20180816/sentiment_analysis_validationset.csv")
data_test_old = pd.read_csv("ai_challenger_sentiment_analysis_testa_20180816/sentiment_analysis_testa.csv")
data_test = pd.read_csv("ai_challenger_sentimetn_analysis_testb_20180816/sentiment_analysis_testb.csv")

print("training.shape:",data.shape)
print("data_validation.shape:",data_validation.shape)
print("data_test_old.shape:",data_test_old.shape)
print("data_test.shape:",data_test.shape)
print("read csv ended...")

read csv started...
training.shape: (105000, 22)
data_validation.shape: (15000, 22)
data_test_old.shape: (15000, 22)
data_test.shape: (200000, 22)
read csv ended...


In [75]:
#print(data.describe())

In [3]:
print(data.head(n=1))

   id                                            content  \
0   0  "吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...   

   location_traffic_convenience  location_distance_from_business_district  \
0                            -2                                        -2   

   location_easy_to_find  service_wait_time  service_waiters_attitude  \
0                     -2                 -2                         1   

   service_parking_convenience  service_serving_speed  price_level  \
0                           -2                     -2           -2   

                ...                 environment_decoration  environment_noise  \
0               ...                                     -2                 -2   

   environment_space  environment_cleaness  dish_portion  dish_taste  \
0                 -2                    -2            -2          -2   

   dish_look  dish_recommendation  others_overall_experience  \
0          1                   -2                          1   


In [4]:
 # id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,price_cost_effective,price_discount,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
#  0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的，不会很甜，中间的照片是糯米的，能食用，真是太高端大气上档次了，还可以买蝴蝶结扎口，送人可以买礼盒。我是先打的卖家电话，加了微信，给卖家传的照片。等了几天，卖家就告诉我可以取货了，去大官屯那取的。虽然连卖家的面都没见到，但是还是谢谢卖家送我这么可爱的东西，太喜欢了，这哪舍得吃啊。""",-2,-2,-2,-2,1,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,-2,1,-2,1,-2

#print(data.head(n=1))
data.shape

(105000, 22)

In [5]:
# print some line to have a look
for index, row in data.iterrows():
    id_=row['id']
    content=row['content']
    label=row[2:]
    print("id_:",id_)
    print("content:",content)
    print("label:",label)
    print("===============================================================")
    if index==5: 
        break

id_: 0
content: "吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的，不会很甜，中间的照片是糯米的，能食用，真是太高端大气上档次了，还可以买蝴蝶结扎口，送人可以买礼盒。我是先打的卖家电话，加了微信，给卖家传的照片。等了几天，卖家就告诉我可以取货了，去大官屯那取的。虽然连卖家的面都没见到，但是还是谢谢卖家送我这么可爱的东西，太喜欢了，这哪舍得吃啊。"
label: location_traffic_convenience                -2
location_distance_from_business_district    -2
location_easy_to_find                       -2
service_wait_time                           -2
service_waiters_attitude                     1
service_parking_convenience                 -2
service_serving_speed                       -2
price_level                                 -2
price_cost_effective                        -2
price_discount                               1
environment_decoration                      -2
environment_noise                           -2
environment_space                           -2
environment_cleaness                        -2
dish_portion                                -2
dish_taste                                  -2
dish

In [33]:
def get_sentiment_analysis_labels(row):
    # 1)location
    location_traffic_convenience = row['location_traffic_convenience']               
    location_distance_from_business_district= row['location_distance_from_business_district'] 
    location_easy_to_find  = row['location_easy_to_find']  
    # 2)service
    service_wait_time  = row['service_wait_time']                         
    service_waiters_attitude = row['service_waiters_attitude']                     
    service_parking_convenience = row['service_parking_convenience']                 
    service_serving_speed  = row['service_serving_speed']  
    # 3)price
    price_level         = row['price_level']                        
    price_cost_effective = row['price_cost_effective']                          
    price_discount  = row['price_discount']     
    # 4)environment
    environment_decoration  = row['environment_decoration']                   
    environment_noise   = row['environment_noise']                        
    environment_space   = row['environment_space']                         
    environment_cleaness     = row['environment_cleaness']  
    # 5)dish
    dish_portion   = row['dish_portion']                              
    dish_taste =row['dish_taste']                                   
    dish_look  = row['dish_look']                                  
    dish_recommendation = row['dish_recommendation']   
    # 6)other
    others_overall_experience  = row['others_overall_experience']                    
    others_willing_to_consume_again   = row['others_willing_to_consume_again']   
    
    label_list=[]
    label_list=[location_traffic_convenience,location_distance_from_business_district,location_easy_to_find, # location
                  service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed, # service
               price_level,price_cost_effective,price_discount, # price
               environment_decoration,environment_noise,environment_space,environment_cleaness, # environment
               dish_portion,dish_taste,dish_look,dish_recommendation, # dish
               others_overall_experience,others_willing_to_consume_again] # other
    label_list=[str(i)+"_"+str(label_list[i]) for i  in range(len(label_list))]
    return label_list 

In [34]:
def convet_to_one_hot(label_list,num_classes=80):
    new_label_list=[0 for i in range(num_classes)]
    for label in label_list:
        new_label_list[label]=1
    return new_label_list

label_list=[0, 4, 8, 12, 19, 20, 24, 28, 32, 39, 43, 47, 51, 55, 59, 62, 64, 68, 75, 76]
label_list_one_hot=convet_to_one_hot(label_list)
print("label_list_one_hot:",label_list_one_hot)

label_list_one_hot: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]


In [36]:
 # print first row
 for index, row in data.iterrows():
    id_=row['id']
    #content=[x for x in jieba.lcut(row['content']) if x.strip() and x!="\""]
    content_raw=row['content']
    content=[content_raw[i] for i in range(len(content_raw)) if content_raw[i].strip() and content_raw[i]!="\"" ]
    sentiment_label_list=get_sentiment_analysis_labels(row)
    print("id_:",id_)
    print("content:",content)
    print("sentiment_label_list:",sentiment_label_list,";length:",len(sentiment_label_list))
    if index==0: 
        break

id_: 0
content: ['吼', '吼', '吼', '，', '萌', '死', '人', '的', '棒', '棒', '糖', '，', '中', '了', '大', '众', '点', '评', '的', '霸', '王', '餐', '，', '太', '可', '爱', '了', '。', '一', '直', '就', '好', '奇', '这', '个', '棒', '棒', '糖', '是', '怎', '么', '个', '东', '西', '，', '大', '众', '点', '评', '给', '了', '我', '这', '个', '土', '老', '冒', '一', '个', '见', '识', '的', '机', '会', '。', '看', '介', '绍', '棒', '棒', '糖', '是', '用', '德', '国', '糖', '做', '的', '，', '不', '会', '很', '甜', '，', '中', '间', '的', '照', '片', '是', '糯', '米', '的', '，', '能', '食', '用', '，', '真', '是', '太', '高', '端', '大', '气', '上', '档', '次', '了', '，', '还', '可', '以', '买', '蝴', '蝶', '结', '扎', '口', '，', '送', '人', '可', '以', '买', '礼', '盒', '。', '我', '是', '先', '打', '的', '卖', '家', '电', '话', '，', '加', '了', '微', '信', '，', '给', '卖', '家', '传', '的', '照', '片', '。', '等', '了', '几', '天', '，', '卖', '家', '就', '告', '诉', '我', '可', '以', '取', '货', '了', '，', '去', '大', '官', '屯', '那', '取', '的', '。', '虽', '然', '连', '卖', '家', '的', '面', '都', '没', '见', '到', '，', '但', '是', '还', '是', '谢', '谢', '卖', '家', '送'

In [3]:
data_traininig_small=data.sample(frac=1.0)
data_validation_small=data_validation.sample(frac=1.0)
data_test_small=data_test_old #[0:1000]
data_test2_small=data_test #[0:1000]

print("data_traininig_small.shape:",data_traininig_small.shape)
print("data_validation_small.shape:",data_validation_small.shape)
print("data_test_small.shape:",data_test_small.shape)
print("data_test2_small.shape:",data_test2_small.shape)

data_traininig_small.shape: (105000, 22)
data_validation_small.shape: (15000, 22)
data_test_small.shape: (15000, 22)
data_test2_small.shape: (200000, 22)


In [4]:
data_big=pd.concat([data_traininig_small,data_validation_small,data_test_small,data_test2_small])#(data_traininig_small, data_test_small, on='key')
print("data_big==>:",data_big.shape)

data_big==>: (335000, 22)


In [45]:
# compute length distribution of inputs and set max_sequence_length; 
# choose a mininum value of max_sequence_length that let 90% of inputs' s length is less of it. actually it cover 93%.
total_length=0
input_length_list=[50,100,150,200,250,300,350,400,500,600,700,1000]
input_length_dict={x:0 for x in input_length_list }
data_big_part=data_big.sample(frac=0.1)

num_example_for_length,_=data_big_part.shape
print("num_example_for_length:",num_example_for_length)
for index, row in data_big_part.iterrows(): # data_traininig_small
    id_=row['id']
    #input_list=[x for x in jieba.lcut(row['content']) if x.strip() and x!="\""]
    content=row['content']
    content=[content[i] for i in range(len(content)) if content[i].strip() and content[i]!="\"" ]
    length_input=len(content)
    
    fixed_len=1000
    for length in input_length_list:
        if length_input<length:
            fixed_len=length
            break
    input_length_dict[fixed_len]=input_length_dict[fixed_len]+1
    total_length+=length_input
    #c_inputs.update(input_list)
    
avg_length=float(total_length) /float(num_example_for_length)
print("avg_length:",avg_length)
print("input_length_dict:",input_length_dict)

input_length_dict_percentage={}
for k,v in input_length_dict.items():
    v=v/num_example_for_length
    input_length_dict_percentage[k]=v
print("input_length_dict_percentage:",input_length_dict_percentage)

# conclusion: most length is between 150-250. average length is 216. if we use max length 350, then we can cover about:  91% of inputs.
# choose a mininum value of max_sequence_length that let 90% of inputs' s length is less of it.
max_sequence_length=0
accumulate_percentage=0
for fixed_length,percentage in input_length_dict_percentage.items():
    accumulate_percentage+=percentage
    if accumulate_percentage>0.9:
        max_sequence_length=fixed_length
        print("max_sequence_length:",max_sequence_length)
        break

num_example_for_length: 33500
avg_length: 336.1846865671642
input_length_dict: {50: 3, 100: 5, 150: 18, 200: 952, 250: 11691, 300: 6506, 350: 4422, 400: 2670, 500: 3060, 600: 1854, 700: 867, 1000: 1452}
input_length_dict_percentage: {50: 8.955223880597016e-05, 100: 0.00014925373134328358, 150: 0.0005373134328358209, 200: 0.028417910447761194, 250: 0.3489850746268657, 300: 0.1942089552238806, 350: 0.132, 400: 0.07970149253731343, 500: 0.09134328358208955, 600: 0.05534328358208955, 700: 0.02588059701492537, 1000: 0.04334328358208955}
max_sequence_length: 600


In [46]:
 # create dict of label to index
value_list=[-2,-1,0,1]
num_aspect=20
label2index={}
count_label=0
for i in range(num_aspect):
    for value in value_list:
        label2index[str(i)+"_"+str(value)]=count_label
        count_label+=1
print("label2index:",label2index)


label2index: {'0_-2': 0, '0_-1': 1, '0_0': 2, '0_1': 3, '1_-2': 4, '1_-1': 5, '1_0': 6, '1_1': 7, '2_-2': 8, '2_-1': 9, '2_0': 10, '2_1': 11, '3_-2': 12, '3_-1': 13, '3_0': 14, '3_1': 15, '4_-2': 16, '4_-1': 17, '4_0': 18, '4_1': 19, '5_-2': 20, '5_-1': 21, '5_0': 22, '5_1': 23, '6_-2': 24, '6_-1': 25, '6_0': 26, '6_1': 27, '7_-2': 28, '7_-1': 29, '7_0': 30, '7_1': 31, '8_-2': 32, '8_-1': 33, '8_0': 34, '8_1': 35, '9_-2': 36, '9_-1': 37, '9_0': 38, '9_1': 39, '10_-2': 40, '10_-1': 41, '10_0': 42, '10_1': 43, '11_-2': 44, '11_-1': 45, '11_0': 46, '11_1': 47, '12_-2': 48, '12_-1': 49, '12_0': 50, '12_1': 51, '13_-2': 52, '13_-1': 53, '13_0': 54, '13_1': 55, '14_-2': 56, '14_-1': 57, '14_0': 58, '14_1': 59, '15_-2': 60, '15_-1': 61, '15_0': 62, '15_1': 63, '16_-2': 64, '16_-1': 65, '16_0': 66, '16_1': 67, '17_-2': 68, '17_-1': 69, '17_0': 70, '17_1': 71, '18_-2': 72, '18_-1': 73, '18_0': 74, '18_1': 75, '19_-2': 76, '19_-1': 77, '19_0': 78, '19_1': 79}


In [42]:
# generate training data by following data format of bert

def process_data(data,target_file):
    target_object=open(target_file,'w')
    print(data.shape)
    for index, row in data.iterrows():
        sentiment_label_list=get_sentiment_analysis_labels(row) 
        content=row['content'].replace("\n","。").replace("\r","。")
        content=[content[i] for i in range(len(content)) if content[i].strip() and content[i]!="\"" ]
        content="".join(content)
        content=re.sub('''"''','',content)
        strings=",".join(sentiment_label_list)+"\t"+content+"\n"
        target_object.write(strings)
        if index%50000==0:print(strings)
    target_object.close()
    

In [48]:
target_file_valid='./data/dev.tsv'
process_data(data_validation_small,target_file_valid)
print("process_data.valid.ended.",len(data_validation_small))
#X_test,_=process_data(data_test_small,data_type='test')
#print("process_data.ended...")


(15000, 22)
0_-2,1_-2,2_-2,3_0,4_-2,5_-2,6_-2,7_1,8_-2,9_-2,10_-2,11_-2,12_-2,13_-2,14_-2,15_0,16_-2,17_-2,18_1,19_0	哎，想当年来佘山的时候，啥都没有，三品香算镇上最大看起来最像样的饭店了。菜品多，有点太多，感觉啥都有，杂都不足以形容。随便点些，居然口味什么的都好还可以，价钱自然是便宜当震惊。元宝虾和椒盐九肚鱼都不错吃。不过近来几次么，味道明显没以前好了。冷餐里面一个凉拌海带丝还可以，酸酸甜甜的。镇上也有了些别的大点的饭店，所以不是每次必来了。对了，这家的生意一如既往的超级好，不定位基本吃不到。不过佘山这边的人吃晚饭很早的，所以稍微晚点去就很空了。

process_data.valid.ended. 15000


In [49]:
target_file_train='./data/train.tsv'
process_data(data_traininig_small,target_file_train)
print("process_data.train.ended.",len(data_traininig_small))

(105000, 22)
0_1,1_1,2_-2,3_-2,4_1,5_1,6_-2,7_-2,8_-2,9_1,10_1,11_1,12_1,13_1,14_1,15_1,16_-2,17_-2,18_1,19_1	非常喜欢去新开的有特色的店品尝，有幸被大众点评网同城聚会抽到黄金替补免费品鉴的机会，我感到十分幸运！红不让台湾特色餐厅位于李沧万达金街1号入口扶梯旁，交通便利，停车方便。店面小资，适合举行小型派对。走近店门，服务员立刻开门迎接并且热情的打招呼。进入店面，环顾四周，鹅黄的灯光下透露出温馨慵懒的气氛，店家匠心独居，屋内的摆设典雅质朴。我们大家围坐在一起开始品尝台湾特色美食，前菜是鸡排沙拉，鸡排外焦里嫩，沙拉酸爽开胃陪的饮品是本店招牌—寒天奶栋，口感丝化。在我们的玩闹嘻戏中，硬菜短上来了——招牌全垒打拼盘，样子可爱，味道独特，被大家迅雷不及掩耳盗铃之势一扫而光。紧接着——等一个人小火锅，西班牙海鲜焗饭，黑椒牛柳焗饭海鲜焗烤嫩鸡排上桌了，食材琳琅满目，令人目不暇接。人多力量大，盘子渐渐见底了，新菜品——普罗旺斯田园披萨，台湾卤肉饭，秘制鸡腿饭，招牌巨蛋烧，阳光手打面，超级霸大热狗震撼来袭。。这里特别提一下普罗旺斯田园披萨，顶上配料五颜六色，芝士浓郁，饼胚酥脆，搭配沙拉和洋葱圈口味绝佳。还有在卤肉饭，鸡腿饭，手打面扮演重要角色的酱料，里面居然搭配了小鱼干，补钙又美味。。环境搭配合理，既有闺密情侣说悄悄话的私密空间，又有三五好友畅饮阔聊的长桌吧台，气氛轻松惬意，互不打扰。。红不让台湾特色餐厅绝对是小资的你餐饮休闲，畅聊发呆的绝佳选择，千万不要错过！

0_-2,1_-2,2_-2,3_-2,4_-2,5_-2,6_-2,7_-2,8_-2,9_-2,10_1,11_-2,12_-2,13_-2,14_-2,15_0,16_0,17_-2,18_1,19_-2	餐前，装冷菜的碟子是仿青瓷的碟子，高脚，可惜是塑料的，近看缺了质感，没有瓷器那种通透，远看挺不错。盛菜的器皿有些也挺有特色的，包厢布置也花了功夫，走古风挂字画。墙纸是青花瓷的，对面墙壁也贴了很多瓷器，有几个还是玲珑瓷，透光看比较好看。比较喜欢他们家的日式茶壶，铁壶，很沉但漂亮，据说冲泡能补铁？菜品味道一般，花样是挺多的，杨枝甘露换个器皿就能换来很大的拍照量，是不是桌桌都有？喜欢那个有小把的酱料器皿，拿着很可

In [50]:
# this is end of genrate training data and validation data for bert
# below is generate pre-train for bert, you can ignore if you don't want to pre-train bert.

In [59]:
print("data_big:",data_big.shape)

data_big: (335000, 22)


In [6]:
 # generate pre-train raw data using data_big
def write_pre_train_doc(data_big,target_path,num_files=30):
    print("data_big.shape:",data_big.shape)
    data_big=data_big.sample(frac=1.0)
    count=0
    data_big
    target_object_list=[]
    for i in range(num_files):
        target_object=open(target_path+"bert_"+str(i)+"_pretrain.txt","w")
        target_object_list.append(target_object)
        
    for index, row in data_big.iterrows():  
        content=row['content']
        content=row['content'].replace("\n","。").replace("\r","。")
        content=[content[i] for i in range(len(content)) if content[i].strip() and content[i]!="\"" ]
        sentence_list=" ".join(content).split("。")
        reminder=count%num_files
        for sentence in sentence_list:
            sentence=sentence.strip()
            target_object_list[reminder].write(sentence+"\n") # each line contains one sentence
        target_object_list[reminder].write(sentence+"\n") # between each document has a blank line
        count=count+1
    for i in range(num_files):
        target_object_list[i].close()
        
print("started...")        
target_path='./PRE_TRAIN_DIR/'
write_pre_train_doc(data_big,target_path)
print("end...")

started...
data_big.shape: (335000, 22)
end...
